<a href="https://colab.research.google.com/github/shelke16/HPC/blob/main/cuda_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install -y build-essential
!apt-get install -y nvidia-cuda-toolkit

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,607 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,863 kB]
Get:13 https://r2u.stat.illinois.edu/ubu

In [2]:
%%writefile matrix.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void matmul(int* A, int* B, int* C) {
    int row = threadIdx.x;
    int col = threadIdx.y;

    if (row < 2 && col < 2) {
        int sum = 0;
        for (int k = 0; k < 2; k++) {
            sum += A[row * 2 + k] * B[k * 2 + col];
        }
        C[row * 2 + col] = sum;
    }
}

int main() {
    // Declare matrices A, B, and C
    int A[2][2] = {{1, 2}, {3, 4}};  // Example 2x2 matrix A
    int B[2][2] = {{5, 6}, {7, 8}};  // Example 2x2 matrix B
    int C[2][2];  // Result matrix C

    int *d_A, *d_B, *d_C;
    int size = 2 * 2 * sizeof(int); // Size of 2x2 matrix

    // Allocate memory on the device (GPU)
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    // Copy data from host to device
    cudaMemcpy(d_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, size, cudaMemcpyHostToDevice);

    // Launch the kernel with a 2x2 block size (one thread per element)
    dim3 threadsPerBlock(2, 2); // 2x2 block of threads
    matmul<<<1, threadsPerBlock>>>(d_A, d_B, d_C);

    // Check for any errors during kernel launch
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA error: %s\n", cudaGetErrorString(err));
        return -1;
    }

    // Copy the result back from device to host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    // Print the result matrix C
    printf("Resulting matrix C:\n");
    for (int i = 0; i < 2; i++) {
        for (int j = 0; j < 2; j++) {
printf("%d ", C[i][j]);
}
printf("\n");
}
// Free device memory
cudaFree(d_A);
cudaFree(d_B);
cudaFree(d_C);
return 0;
}

Writing matrix.cu


In [3]:
!nvcc matrix.cu -o matrix

In [4]:
!./matrix

CUDA error: no CUDA-capable device is detected


In [16]:
%%writefile matrix.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

__global__ void add(int* A, int* B, int* C, int size) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x; // Calculate thread ID

    // Check if the thread ID is within the vector size
    if (tid < size) {
        C[tid] = A[tid] + B[tid];
    }
}

__global__
void multiply(int* A, int* B, int* C, int size) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < size && col < size) {
        int sum = 0;
        for (int i = 0; i < size; i++) {
            sum += A[row * size + i] * B[i * size + col];
        }
        C[row * size + col] = sum;
    }
}

void initializeVector(int* vector, int size) {
    for (int i = 0; i < size; i++) {
        vector[i] = rand() % 10;
    }
}

void initializeMatrix(int* matrix, int size) {
    for (int i = 0; i < size * size; i++) {
        matrix[i] = rand() % 10;
    }
}

void printVector(int* vector, int size) {
    for (int i = 0; i < size; i++) {
        printf("%d ", vector[i]);
    }
    printf("\n");
}

void printMatrix(int* matrix, int size) {
    for (int row = 0; row < size; row++) {
        for (int col = 0; col < size; col++) {
            printf("%d ", matrix[row * size + col]);
        }
        printf("\n");
    }
    printf("\n");
}

int main() {
    int N = 4;

    // Vector addition
    int* A, * B, * C;
    int vectorSize = N;
    size_t vectorBytes = vectorSize * sizeof(int);

    A = (int*)malloc(vectorBytes);
    B = (int*)malloc(vectorBytes);
    C = (int*)malloc(vectorBytes);

    initializeVector(A, vectorSize);
    initializeVector(B, vectorSize);

    printf("Vector A: ");
    printVector(A, N);
    printf("Vector B: ");
    printVector(B, N);

    int* X, * Y, * Z;
    cudaMalloc(&X, vectorBytes);
    cudaMalloc(&Y, vectorBytes);
    cudaMalloc(&Z, vectorBytes);

    cudaMemcpy(X, A, vectorBytes, cudaMemcpyHostToDevice);
    cudaMemcpy(Y, B, vectorBytes, cudaMemcpyHostToDevice);

    int threadsPerBlockVec = 256;
    int blocksPerGridVec = (N + threadsPerBlockVec - 1) / threadsPerBlockVec;

    add<<<blocksPerGridVec, threadsPerBlockVec>>>(X, Y, Z, N);

    cudaMemcpy(C, Z, vectorBytes, cudaMemcpyDeviceToHost);

    printf("Addition: ");
    printVector(C, N);

    free(A);
    free(B);
    free(C);

    cudaFree(X);
    cudaFree(Y);
    cudaFree(Z);

    // Matrix multiplication
    int* D, * E, * F;
    int matrixSize = N;
    size_t matrixBytes = matrixSize * matrixSize * sizeof(int);

    D = (int*)malloc(matrixBytes);
    E = (int*)malloc(matrixBytes);
    F = (int*)malloc(matrixBytes);

    initializeMatrix(D, matrixSize);
    initializeMatrix(E, matrixSize);

    printf("\nMatrix D: \n");
    printMatrix(D, matrixSize);

    printf("Matrix E: \n");
    printMatrix(E, matrixSize);

    int* M, * NMat, * O;
    cudaMalloc(&M, matrixBytes);
    cudaMalloc(&NMat, matrixBytes);
    cudaMalloc(&O, matrixBytes);

    cudaMemcpy(M, D, matrixBytes, cudaMemcpyHostToDevice);
    cudaMemcpy(NMat, E, matrixBytes, cudaMemcpyHostToDevice);

    int threadsPerBlockMat = 2;
    int blocksPerGridMat = (matrixSize + threadsPerBlockMat - 1) / threadsPerBlockMat;

    dim3 threadsMat(threadsPerBlockMat, threadsPerBlockMat);
    dim3 blocksMat(blocksPerGridMat, blocksPerGridMat);

    multiply<<<blocksMat, threadsMat>>>(M, NMat, O, matrixSize);

    cudaMemcpy(F, O, matrixBytes, cudaMemcpyDeviceToHost);
    printf("Multiplication: \n");
    printMatrix(F, matrixSize);

    free(D);
    free(E);
    free(F);

    cudaFree(M);
    cudaFree(NMat);
    cudaFree(O);

    return 0;
}

Overwriting matrix.cu


In [17]:
!./matrix

Vector A: 3 6 7 5 
Vector B: 3 5 6 2 
Addition: 0 0 0 0 

Matrix D: 
9 1 2 7 
0 9 3 6 
0 6 2 6 
1 8 7 9 

Matrix E: 
2 0 2 3 
7 5 9 2 
2 8 9 7 
3 6 1 2 

Multiplication: 
2014691584 31576 -839342928 22242 
0 0 0 0 
2015095536 31576 -839344224 22242 
0 0 -839343056 22242 

